In [ ]:
import pandas as pd
import numpy as np
from xgboost import XGBClassifier

# Load data
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Target variable
y = train['Transported'].astype(int)  # True/False to 1/0

# Combine train and test for consistent preprocessing
test['Transported'] = np.nan
full = pd.concat([train, test], sort=False)

# Fill missing values and encode categorical features
full['HomePlanet'] = full['HomePlanet'].fillna('Unknown')
full['CryoSleep'] = full['CryoSleep'].fillna(False).astype(int)
full['Cabin'] = full['Cabin'].fillna('Unknown/Unknown/Unknown')
full['Destination'] = full['Destination'].fillna('Unknown')
full['Age'] = full['Age'].fillna(full['Age'].median())
full['VIP'] = full['VIP'].fillna(False).astype(int)
for col in ['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']:
    full[col] = full[col].fillna(0)

# Split Cabin into 3 features
full[['CabinDeck', 'CabinNum', 'CabinSide']] = full['Cabin'].str.split('/', expand=True)
full['CabinDeck'] = full['CabinDeck'].astype('category').cat.codes
full['CabinSide'] = full['CabinSide'].astype('category').cat.codes

# Encode categorical variables
for col in ['HomePlanet', 'Destination']:
    full[col] = full[col].astype('category').cat.codes

# Feature list
features = [
    'HomePlanet', 'CryoSleep', 'Destination', 'Age', 'VIP',
    'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
    'CabinDeck', 'CabinSide'
]

# Split back to train and test
X = full.loc[full['Transported'].notnull(), features]
X_test = full.loc[full['Transported'].isnull(), features]

# Train model
model = XGBClassifier(n_estimators=300, max_depth=5, learning_rate=0.03, use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X, y)

# Predict
test_preds = model.predict(X_test)
test['Transported'] = test_preds.astype(bool)

# Save submission
test[['PassengerId', 'Transported']].to_csv('submission.csv', index=False)

C:\Users\Wilde\AppData\Local\Temp\ipykernel_22948\1942404488.py:18: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full['CryoSleep'] = full['CryoSleep'].fillna(False).astype(int)
C:\Users\Wilde\AppData\Local\Temp\ipykernel_22948\1942404488.py:22: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  full['VIP'] = full['VIP'].fillna(False).astype(int)
c:\Users\Wilde\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:51:21] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label